<a href="https://colab.research.google.com/github/naufalhisyam/TurbidityPrediction-thesis/blob/main/model_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
!pip install tensorflow-addons
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split

In [ ]:
!git clone https://github.com/naufalhisyam/TurbidityPrediction-thesis.git
os.chdir('/content/TurbidityPrediction-thesis') 
!ls

**PREPARING DATASET**

In [ ]:
images = pd.read_csv(r'Datasets\0degree\0degInfo.csv') #load dataset info
train_df, test_df = train_test_split(image_df, train_size=0.8, shuffle=True, random_state=1) #Split into train and test set

In [ ]:
train_generator = keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    validation_split=0.2
)

test_generator = keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

**CREATING THE MODEL**

Defining

In [2]:
def get_model():
    #Create model
    base_model = keras.applications.ResNet50(include_top=False, weights=None, 
                                                input_shape=(224, 224, 3), pooling='avg')
    x = base_model.output
    prediction = keras.layers.Dense(1, activation="linear")(x)
    model = keras.Model(inputs = base_model.input, outputs = prediction)

    #Compile the model
    opt = keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200)
    model.compile(loss="mse", optimizer=opt,
                metrics=['mse', 'mape', tfa.metrics.RSquare(name="R2", y_shape=(1,))])
    
    return model

Training

In [ ]:
history = model.fit(train_images,
    validation_data=val_images,
    epochs=200, batch_size=8
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True
        )
    ],
)

In [ ]:
name = 'tes'
model.save(f'saved_model/{name}') #Saves the trained model

Model Summary

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "my_first_model.png", show_shapes=False)